In [3]:
import jupyter_black

jupyter_black.load()

import pandas as pd
from src.utils.connections import load_service_account
import nfl_data_py as nfl

sa = load_service_account()

In [11]:
HFA = 1.5
AVG_POINTS = 21.5
SEASON, WEEK = 2023, 14

In [4]:
sheet = sa.open("NFL Power Ratings - 2023")

In [13]:
work_sheet = sheet.worksheet(f"Wk {WEEK}")
df = pd.DataFrame(work_sheet.get_all_records())

In [14]:
df

,team,ovr,off,def,off_notes,def_notes
0,SF,7.6,4.7,2.9,"deebo game, offense is a truck when healthy","pressured hurts a lot, plays so physical, gree..."
1,BAL,5.5,2.5,3.0,bye,bye
2,MIA,5.5,5.3,0.2,"properly rated, tyreek opoy, achane back and p...",
3,DAL,5.1,2.6,2.5,"mccarthy bad playcalling, dak still dealing",secondary got burned by good receivers
4,PHI,5.0,4.4,0.6,"out physicalled but coming off short rest, won...",bad secondary
5,KC,4.8,2.9,1.9,big rashee game,did not show up in GB
6,BUF,3.6,3.4,0.2,bye,bye
7,JAX,2.8,2.4,0.4,,could you please not let Jake Browning score 3...
8,HOU,1.8,2.0,-0.2,"running game not that great, stroud is money, ...","stingley was impressive, pass rush was solid"
9,DET,1.7,3.1,-1.4,"running game is impressive, decent goff game","nothing inspiring, very gettable"


In [15]:
sched = nfl.import_schedules([SEASON])
sched = (
    sched[sched["week"] == WEEK][["game_id", "away_team", "home_team"]]
    .copy()
    .set_index("game_id")
)
sched.head()

,away_team,home_team
game_id,,
2023_14_NE_PIT,NE,PIT
2023_14_TB_ATL,TB,ATL
2023_14_LA_BAL,LA,BAL
2023_14_DET_CHI,DET,CHI
2023_14_IND_CIN,IND,CIN


In [16]:
predict_home_score = lambda row: AVG_POINTS + row.home_off - row.away_def + HFA / 2
predict_away_score = lambda row: AVG_POINTS + row.away_off - row.home_def - HFA / 2

In [19]:
# guess the lines
gtl = (
    pd.merge(
        sched,
        df[["team", "ovr", "off", "def"]],
        left_on="away_team",
        right_on="team",
    )
    .drop(columns=["team"])
    .rename(columns={"ovr": "away_ovr", "off": "away_off", "def": "away_def"})
)
gtl = (
    pd.merge(
        gtl,
        df[["team", "ovr", "off", "def"]],
        left_on="home_team",
        right_on="team",
    )
    .drop(columns=["team"])
    .rename(columns={"ovr": "home_ovr", "off": "home_off", "def": "home_def"})
)
gtl["hfa"] = HFA
gtl["pred_away_score"] = gtl.apply(predict_away_score, axis=1)
gtl["pred_home_score"] = gtl.apply(predict_home_score, axis=1)
gtl["spread"] = gtl["pred_home_score"] - gtl["pred_away_score"]
gtl["total"] = gtl["pred_home_score"] + gtl["pred_away_score"]
gtl = gtl[
    [
        "away_team",
        "home_team",
        "spread",
        "total",
        "hfa",
        "pred_away_score",
        "pred_home_score",
        "away_ovr",
        "home_ovr",
    ]
].copy()
gtl["game_id"] = gtl.apply(
    lambda row: f"{SEASON}_{WEEK}_{row.away_team}_{row.home_team}", axis=1
)
gtl = gtl.set_index("game_id")
gtl

,away_team,home_team,spread,total,hfa,pred_away_score,pred_home_score,away_ovr,home_ovr
game_id,,,,,,,,,
2023_14_NE_PIT,NE,PIT,6.2,35.9,1.5,14.85,21.05,-6.2,-1.5
2023_14_TB_ATL,TB,ATL,3.3,41.8,1.5,19.25,22.55,-3.6,-1.8
2023_14_LA_BAL,LA,BAL,7.5,43.4,1.5,17.95,25.45,-0.5,5.5
2023_14_DET_CHI,DET,CHI,-1.4,45.3,1.5,23.35,21.95,1.7,-1.2
2023_14_IND_CIN,IND,CIN,-0.6,42.9,1.5,21.75,21.15,-0.9,-3.0
2023_14_JAX_CLE,JAX,CLE,-2.7,42.0,1.5,22.35,19.65,2.8,-1.4
2023_14_CAR_NO,CAR,NO,5.9,42.0,1.5,18.05,23.95,-6.0,-1.6
2023_14_HOU_NYJ,HOU,NYJ,-4.3,36.2,1.5,20.25,15.95,1.8,-4.0
2023_14_MIN_LV,MIN,LV,-1.3,41.0,1.5,21.15,19.85,-1.1,-3.9
